### Canned laughter identification using pre-trained models

In this notebook, we will use model(s) trained on friends laughter data
to detect laughter in any media file

In [ ]:
import sys
sys.path.append('../utils/')

In [ ]:
# local imports
import utils
import episode
import color
import stats
import modelbuilder
# stdlib and package imports
import numpy as np
import pandas as pd
from pathlib import Path 
from matplotlib import pyplot as plt
from collections import Counter
# keras and ML imports
from keras.models import Sequential, Model, model_from_yaml
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as sknormalize
from sklearn.metrics import confusion_matrix
from imblearn import under_sampling

### Load an existing model

In [ ]:
model = modelbuilder.build_laugh_model()
model.load_weights(filepath='task:per-season-split-ckpt.hdf5')
model = modelbuilder._compile_binary(model)

### Mark an entire episode with labels

In [ ]:
decoded, preds = episode.detect_in_episode(episode='friends-s03-e09', model=model, precision=2,
                                           algorithms=['threshold', 'hmm'], params=dict(n=3, t=.6))

In [ ]:
df = pd.DataFrame(decoded)

In [ ]:
annot = episode.load_annotations(episode='friends-s03-e09')['laughter']

In [ ]:
def in_any_patch(time, annot=annot):
    for patch in annot:
        if time in range(*patch):
            break
    else:
        return False
    return True

In [ ]:
df['raw_preds'] = [int(pred > .5) for pred in preds]
df['y_true_annot'] = df['timestamp'].apply(lambda t: int(in_any_patch(t)))
# df['positive_label'] = df[df['threshold'] == 1]['timestamp'].apply(lambda x: in_any_patch(x))
# df['negative_label'] = df[df['threshold'] == 0]['timestamp'].apply(lambda x: not in_any_patch(x))

In [ ]:
df[:64] # INFO: {positive,negative}_label = {True,False}: indicates whether it's a true positive or false positive or ...

In [ ]:
# do these algorithms even produce their own result (False = YES, they produce something different)
all(df['hmm'] == df['raw_preds']), all(df['threshold'] == df['raw_preds'])

In [ ]:
[*zip(*[[(key,val) for val in decoded[key]] for key in decoded])]